In [1]:
from selenium.webdriver.chrome.options import Options 
from selenium import webdriver
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import requests
import random
import time

In [2]:
year_to_pull='2019'
sched_url='https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/'+ year_to_pull +'/league/00_full_schedule_week.json' #edit later and make it dynamic
print(sched_url)
response = requests.get(sched_url)
data=response.json() #convert the response in to a JSON dictionary
schedule=data['lscd'] #get all the games by month


https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/2019/league/00_full_schedule_week.json


In [3]:
existing_games=[]
try:
    df_check=pd.read_csv(year_to_pull+'_nba_gamelogs.csv',dtype={'gid': object}) #open the existing database for the selected year. Make sure to set 'gid' to object so leading zeroes are not removed
    existing_games=list(set(df_check.gid)) #get a list of the games already in the database. Use a tuple to only keep the unique games

except:
    print('no csv')

In [4]:
from datetime import datetime #import the datetime module
game_mon=[schedule[x]['mscd'] for x in range(0,len(schedule))] #filter down to the subjset of the JSON with the gameIDs

game_list=[] #a list contains all of the months in the year, the individual gameIDs and the date of the game
temp =[]

for i in range(0,len(game_mon)): #gam_mon is an array of all the months in the json
    num_games=len(game_mon[i]['g']) #number of games in a month
    if int(year_to_pull)>2016: #after 2016 need to exclude preseason
        temp.append([{game_mon[i]['g'][a]['gid']:datetime.strptime(game_mon[i]['g'][a]['gdte'],'%Y-%m-%d').date()} for a in range(0,num_games) if game_mon[i]['g'][a]['gweek'] != None]) #list of gameIds and the game_dates converted into python dates (no datetime)
    else: #preseason games do not appear to be captured before 2017
        temp.append([{game_mon[i]['g'][a]['gid']:datetime.strptime(game_mon[i]['g'][a]['gdte'],'%Y-%m-%d').date()} for a in range(0,num_games)]) #list of gameIds and the game_dates converted into python dates (no datetime)

#temp =list(filter(len, temp)) #remove any empty elements
game_list = [item for sublist in temp for item in sublist] #flatten the list to get a list of game_ids and the the game dates

In [5]:
schedule[0]['mscd']['g'][0]['gid']

'0011900001'

In [6]:
from collections import ChainMap
game_list=dict(ChainMap(*game_list)) #convert the list to one dictionary

In [7]:
from datetime import datetime #import the datetime module
max_date=datetime.date(datetime.now()) #get the current date and set it the upper bound for the query.

In [8]:
game_list_key = [key for key,val in game_list.items() if val < max_date and key not in existing_games] #identify the games for the query

In [9]:
game_list_key=[item for item in game_list_key if item not in existing_games]
print(len(game_list_key),game_list_key)

4 ['0021900370', '0021900369', '0021900368', '0021900367']


In [10]:
def create_df(data):
    from datetime import date
    gm_id=data['g']['gid'] #get the game ID
    #access the dictionary keys for the visitors and home teams
    vt_pl_sel=data['g']['vls']['pstsg'] #visiting team player stats
    hm_pl_sel=data['g']['hls']['pstsg'] #home team player stats
    
    #visitors_df
    v_df=pd.DataFrame.from_dict(vt_pl_sel)
    v_df['gid']=gm_id #add game id
    v_df['hm_away']='A' #set as away team
    v_df['team']=data['g']['vls']['ta'] #get visitor team name
    v_df['game_date']=data['g']['gdte'] #get the date of the game


    #home df
    h_df=pd.DataFrame.from_dict(hm_pl_sel)
    h_df['gid']=gm_id #add game id
    h_df['hm_away']='H' #set as home team
    h_df['team']=data['g']['hls']['ta'] #get home team name
    h_df['game_date']=data['g']['gdte'] #get the date of the game
    
    box_df=v_df.append(h_df,ignore_index=True) #merge home and away dateframes
    box_df['full_name']=box_df.fn+' '+box_df.ln #add full name field
    box_df['epoch']= data['g']['mid'] #get epoch time
    box_df['full_dt'] = pd.to_datetime(box_df['epoch'], unit = 'ms')
    return box_df #return the df for that day

In [11]:
#loop through games and create dataframe

start_time = time.time()

box_df=pd.DataFrame() #empty the old dataframe
for i in game_list_key: #loop through all of the games 
    game_url='https://data.nba.com/data/10s/v2015/json/mobile_teams/nba/' + year_to_pull + '/scores/gamedetail/'+str(i)+'_gamedetail.json'
    response = requests.get(game_url)
    data=response.json() #convert the response in to a JSON dictionary
    box_df=box_df.append(create_df(data),ignore_index=True)
    box_df['season']=(str(year_to_pull)+'-'+str((int(year_to_pull)+1)))

print("--- %s seconds ---" % (time.time() - start_time))

    
box_df.gid.dtype
#ttps://stats.nba.com/stats/boxscoretraditionalv2?EndPeriod=10&EndRange=31800&GameID=0021900366&RangeType=0&Season=2019-20&SeasonType=Regular+Season&StartPeriod=1&StartRange=0

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


--- 1.5589003562927246 seconds ---


dtype('O')

In [12]:
import os.path
from os import path
filename=year_to_pull+'_nba_gamelogs.csv'

if os.path.isfile(filename):
    box_df.to_csv(year_to_pull+'_nba_gamelogs.csv',index=False, mode='a',header=False)
else:
    box_df.to_csv(year_to_pull+'_nba_gamelogs.csv',index=False)

In [13]:
box_df.head()

,ast,blk,blka,court,dreb,epoch,fbpts,fbptsa,fbptsm,fga,...,season,sec,status,stl,team,tf,totsec,tov,tpa,tpm
0,2,0,0,1,3,1576216459520,0,0,0,7,...,2019-2020,2,A,0,POR,0,1802,1,5,1
1,1,2,0,0,9,1576216459520,5,3,2,16,...,2019-2020,42,A,2,POR,0,2202,2,5,2
2,2,4,1,0,8,1576216459520,0,0,0,21,...,2019-2020,6,A,0,POR,1,2106,0,1,1
3,2,0,0,0,2,1576216459520,0,1,0,11,...,2019-2020,35,A,0,POR,0,2195,3,7,3
4,11,0,0,0,1,1576216459520,0,2,0,16,...,2019-2020,33,A,1,POR,0,2253,2,9,2


In [16]:
box_df.columns

Index(['ast', 'blk', 'blka', 'court', 'dreb', 'epoch', 'fbpts', 'fbptsa',
       'fbptsm', 'fga', 'fgm', 'fn', 'fta', 'ftm', 'full_dt', 'full_name',
       'game_date', 'gid', 'hm_away', 'ln', 'memo', 'min', 'num', 'oreb', 'pf',
       'pid', 'pip', 'pipa', 'pipm', 'pm', 'pos', 'pts', 'reb', 'season',
       'sec', 'status', 'stl', 'team', 'tf', 'totsec', 'tov', 'tpa', 'tpm'],
      dtype='object')